<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/product_category%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    classification_report,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/lg/train.csv')

In [ ]:
data.head()

In [ ]:
#other또는 etc.으로갈것>>>other도 etc.으로 가자
etc_list=['AI/Machine Learning | Antennas, Transmitters and Towers | Audience Measurement | Cameras and Lenses',
            'Architect , We are Meeting for Enqiry Generation ( This is not a Inquiry)','Category','Channel Partner done FTS Details in System','LG SuperSign CMS',
          'other포함으로','Thin Clients','Zero Clients','a definir','ahu','aircare','allinone_rmk','autre','corpouh5f','corpuh5f-','ctv','ems,audio/video','energy storage system','error',
          'essential series','gsca046','gscd046','gscd100','high inch 86 / 98 or 110','htv','id','inne']
#signage라는게 매우매우애매하다. 내생각엔 signage최소 signage끼리묶자. edu는 모델명들이 signage쓰는거인듯 구글링결과
signage=['signage포함으로','education createboard','fhd series','high brightness포함']

#tv로냅둘지 모니터로갈지 디스플레이로갈지
tv=['TV Monitors','comercial tv','commercial tv포함으로']
cloud_device=['cloud포함으로']
medical_display=['Medical Display포함으','Clinical Review','Diagnostic 포함으로','X-ray포함으로','Monitor De Diagnostico','Surgical Monitors']
projector=['project포함으로','ProBeam','bu50nst']
#그냥 monitor랑 display로 나뉜거랑 큰 차이없긴한데,
monitor=['Ergonomic monitors','IPS Monitors','Monitors for graphics applications & video editing','Monitors with USB-C connection','UHD 4K Monitors',
         '32UN880포함으로','34WN780포함으','UltraWide™ Monitors','computer monitors']
#hotel tv
hotel=['hospitality','hoteleria_us670h',]
#lg one:quick series로갈것
onequick=['SC-00DA','aio포함으로','idb']
heating=['a thermodynamic water heater','aquecimento','calefacción']
vrf=['all lg vrf systems']
chiller=['chiller포함으로']
#가정용에어컨쪽
rac=['ac rumah','aire acondicionado residencial','ar condicionado residencial','cac포함으로','climatiseur résidentiel']
#'accessories'는 건들지말아보는게?

#interactive digital board는 광범위긴함
interactive=['collaboration displays']

In [ ]:
#interactive digital board인것 modelname>>> 55VSM5J,55CT5WJ,75TR3BF-B,GSCC066,43HT3WJ,75TC3D,LG SuperSign CMS,86TR3BF-B,86TR3E,LAS039DB9-V,LSAB009
#OneQuick이 들어간게 sub에 있으면 interactive digital board인가?중요 Compact Series,LG MAGNIT,Video Wall
#그렇지않았다. quick이 들어간거 중에선 one quick series가 오히려 더 많았다. 그러면 onequick을 어떻게 ???

#vrf에는 split계열이 다 들어있는데(에어컨) 정리어떻게할지..
#video wall> sub에 video wall 이 있거나.. modelname으로 구별하려했는데, interactive에도jhk19 있던 모델들
#product_category 위에 숫자는 모니터를 뜻함. 다 건들필요있을 수도. nan이 19374개
#value count를 보면 Interactive Digital Board는 모니터, vrf는 에어컨쪽 multisplit은 에어컨 각방조절? video wall은 큰광고판화면같
#lg signage는 애매, single-split도 에어컨 rac도 에어컨쪽 chiller 에어컨부품? 냉매 one quick은 모니터?
#data['product_subcategory']와 data['product_category'] 를 비교해서 nan을 채울까
#data['product_modelname']랑도 연관 지으려면 지을수있

In [ ]:
##다시생각하자

#32un880 UltraFine은 False만 있음. 모니터긴한데 일단 분류안하기로?
#pol들어간것들은 코드상정보x
#32lq621cbsb.awz도 코드상정보x
#awz들어간 많은것들 중엔 hotel tv하나있긴함, bwz도 호텔티비하나
#43us는 호텔티비가 대다수
#49uh는 standard signage
#49vl은 video wall signage
#55svh는 video wall signage
#55tc3d는 interactive signage,+ 互動式顯示屏가 interactive signage인듯
#55vm은 video wall signage
#86uh는 standard signage 98uh도 동일
#AI, all cloud는 다 false
commercialtv_list=['commercial tv']
monior_list=['monior/monitor']#monior/monitor
onequick_list=['43HT3WJ','55CT5WJ','SC-00DA','lg one:quick']#43ht는 fal 'lg one:quick series'로 등
aio_list=['aio']
trash_list=['AI/Machine','All Cloud','All Projectors','Architect','Category','Channel','X-ray','Client','corp']#이건 false고 어디 넣기도 애매한것들 other에 넣는게 낫나?
monitor_list=['31HN713D','32un880','34WN780','43HT3WJ','49uh / 49xf','28MQ','27Q']#False 비율높은 모니터?
interactive_list=['55tc3d']#55tc3d는 true
signage_list=['49vl5f','49vl5g','55svh7f','55vm5e-a','55vm5j-h','65EP5G','86uh5f','98uh5e','Signage','SuperSign','gsca046','gscd']#49vl5f는 fal, gsca는 true이긴한데 따로 빼면안되지않나
hotel_tv=['awz','43us660h','bwz']#awz,bwz는 hoteltv임. True 비율이 높음
videowall_list=['video wall']
medicaldisplay_list=['medical display','Clinical Review Monitors','Diagnostic','Surgical']#28mq780는 대부분 medical displays가 많음
monitor_list=['Monitor','21HQ513D','32UN8','Ergonomic','IPS','tv','commercial']
projector_list=['projector']
vrf_list=['vrf']#vrf만으로할까?
chiller_list=['chiller']#chiller랑 vrf, split 관계있는듯
heating_list=['water','heater']
ac_list=['ac rumah','ahu','aire acondicionado residencial','climatiseur résidentiel']#에어컨
aircare_list=['aircare']
probeam_list=['bu50nst']
brightness_list=['brightness']#high brightness로 등록
顯示屏_list=['顯示屏']
multi_split=['multi split']
#autre 는 other일지 지켜보자

category_lists = [videowall_list,medicaldisplay_list,monitor_list,projector_list,vrf_list]
category_names = ['video wall signage','medical display','monitor','projector','vrf']

for i in range(len(category_lists)):
    for category in category_lists[i]:
        data.loc[data['product_category'].str.contains(category, case=False, na=False), 'product_category'] = category_names[i]

In [ ]:
data[['product_category', 'product_subcategory','product_modelname']]

In [ ]:
data.loc[data['product_subcategory'].isna(), 'product_subcategory'] = data['product_modelname']

In [ ]:
data.loc[data['product_category'].isna(), 'product_category'] = data['product_subcategory']

In [ ]:
data['product_category'].isnull().sum()

In [ ]:
set(data['product_category'])